In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
from tqdm import tqdm

In [2]:
def driver():
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    s = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    #options.add_argument("headless")
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--window-size=1920,1080")
    options.binary_location = '/Applications/Brave Browser.app/Contents/MacOS/Brave Browser'
    return webdriver.Chrome(service=s, options=options)
    
def linkedin_login():
    mail = 'praisedbethelord69@gmail.com'
    pw = '3m9MLq7rbPgnqLs'
    driver.get('https://www.linkedin.com')
    username = driver.find_element(By.ID, 'email-or-phone').send_keys(mail)
    password = driver.find_element(By.ID, 'password').send_keys(pw)
    login_button = driver.find_element_by_class_name("sign-in-form__submit-btn--full-width")

    login_button.click()

In [3]:
def data_analyst():
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    s = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    # options.add_argument("headless")
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--window-size=1920,1080")
    options.binary_location = '/Applications/Brave Browser.app/Contents/MacOS/Brave Browser'
    driver = webdriver.Chrome(service=s, options=options)
    mail = 'praisedbethelord69@gmail.com'
    pw = '3m9MLq7rbPgnqLs'
    driver.get('https://www.linkedin.com')
    username = driver.find_element(By.ID, 'session_key').send_keys(mail)
    password = driver.find_element(By.ID, 'session_password').send_keys(pw)
    time.sleep(10)

    titles = []
    companies = []
    locations = []
    modalities = []
    descriptions = []

    i = 0
    keep_scrolling = True

    while keep_scrolling:
        driver.get(f"https://www.linkedin.com/jobs/search/?currentJobId=3658273235&geoId=105646813&keywords=data%20analyst&location=Spain&refresh={i}")
        time.sleep(random.randint(3, 5))
        all_listings = []

        listings = driver.find_elements(By.CSS_SELECTOR, ".job-card-list__title")

        if set(listings) == set(all_listings):
            keep_scrolling = False
        else:
            new_listings = list(set(listings) - set(all_listings))
            for listing in new_listings:
                all_listings.append(listing)
                try:
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth'});", listing)
                    time.sleep(random.randint(1, 3))
                    listing.click()
                    time.sleep(random.randint(2, 5))
                    page_source = driver.page_source
                    soup = BeautifulSoup(page_source, "html.parser")
                    try:
                        title = driver.find_element(By.XPATH, "//*[starts-with(@id, 'ember')]/h2").text
                        if 'data' in title.lower() or 'datos' in title.lower():
                            titles.append(title)
                            try:
                                company = driver.find_element(By.XPATH, "//*[contains(@class, 'ember-view t-black t-normal')]").text
                                companies.append(company)
                            except IndexError:
                                companies.append('Unknown')
                            try:
                                location = soup.select('span.jobs-unified-top-card__bullet')[0].get_text().strip()
                                locations.append(location)
                            except IndexError:
                                locations.append('Unknown')
                            try:
                                modality = soup.select('span.jobs-unified-top-card__workplace-type')[0].get_text()
                                modalities.append(modality)
                            except IndexError:
                                modalities.append('Unknown')
                            try:
                                description = soup.select('#job-details > span')[0].get_text()
                                descriptions.append(description)
                            except IndexError:
                                descriptions.append('Unknown')
                            
                    except NoSuchElementException:
                        driver.save_screenshot(f"{random.randint(0, 100)}.png")
                except StaleElementReferenceException:
                    driver.save_screenshot(f"{random.randint(0, 100)}.png")
            time.sleep(random.randint(1, 2))
        i += 1

    driver.quit()
    df = pd.DataFrame({'title': titles, 'company': companies, 'location': locations,
                       'modality': modalities, 'description': descriptions})
    return df